<a href="https://colab.research.google.com/github/juliowatanabe/Alura_Google_AI/blob/main/Alura_Google_AI_Taroh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este programa ...

In [237]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [213]:
# Configurações iniciais
import google.generativeai as genai
import numpy as np
import pandas as pd
import random
# Importa o módulo userdata do google.colab para acessar o meu API_Key armazenado no Secrets
from google.colab import userdata

api_key = userdata.get("JKW_API_Key")

GOOGLE_API_KEY= api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [238]:
# Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [239]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [240]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [241]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [242]:
carta = random.randint(1, 21)
response = model.generate_content(f"qual é a carta {carta} do Taroh.")
response.text

'## A Carta 16 do Tarô: A Torre\n\nA carta 16 do Tarô é **A Torre**. Ela é uma das cartas mais temidas e mal compreendidas do baralho, frequentemente associada a mudanças repentinas, caos e destruição. No entanto, é importante lembrar que a destruição que ela representa é muitas vezes necessária para o crescimento e a transformação.\n\n**Simbolismo da carta:**\n\n* **A torre:** Representa estruturas rígidas, crenças limitantes e falsas ilusões que precisam ser derrubadas para que possamos construir algo novo e melhor.\n* **Raio:** Simboliza a força da mudança repentina e inesperada, uma intervenção divina ou um despertar espiritual.\n* **Pessoas caindo:** Representam o ego e o apego a coisas materiais que são destruídas durante o processo de transformação.\n* **Coroas caindo:** Simbolizam a perda de poder, status ou controle.\n\n**Interpretações da carta:**\n\n* **Mudança repentina e inesperada:** A Torre pode indicar uma reviravolta na vida, como uma perda de emprego, o fim de um rela

In [244]:
significado = model.generate_content(f'Consulte o significado da Carta do Taroh {response}')

In [245]:
significado.text

'A resposta do modelo de linguagem forneceu uma interpretação abrangente da carta da Torre no Tarô. Aqui estão os pontos principais:\n\n**Simbolismo:**\n\n* **Destruição:** A Torre representa a demolição de estruturas rígidas, crenças limitantes e falsas ilusões.\n* **Mudança repentina:** O raio simboliza a natureza inesperada e poderosa da mudança, muitas vezes além do nosso controle.\n* **Perda e Libertação:** As figuras caindo e as coroas representam a perda de apegos materiais, poder e controle, levando a uma eventual libertação.\n\n**Interpretações:**\n\n* **Reviravoltas:** A Torre prenuncia mudanças abruptas e inesperadas na vida, como o fim de um relacionamento, perda de emprego ou crises pessoais.\n* **Verdades Reveladas:** A carta expõe verdades ocultas, forçando-nos a confrontar a realidade e abandonar ilusões.\n* **Transformação:** A Torre sinaliza um despertar espiritual, onde velhos padrões são quebrados para dar lugar a uma nova consciência.\n* **Renovação:** Apesar do ca

In [246]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [247]:
title = "Taroh"
sample_text = significado.text

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

#print(embeddings)

In [248]:
DOCUMENT1 = {
    "Título": "Taroh",
    "Conteúdo": significado.text}

documents = [DOCUMENT1]

In [249]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Taroh,A resposta do modelo de linguagem forneceu uma...


In [250]:
model = "models/embedding-001"

In [251]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [252]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
#df

In [253]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [254]:
consulta = "qual é o siginificado"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A resposta do modelo de linguagem forneceu uma interpretação abrangente da carta da Torre no Tarô. Aqui estão os pontos principais:

**Simbolismo:**

* **Destruição:** A Torre representa a demolição de estruturas rígidas, crenças limitantes e falsas ilusões.
* **Mudança repentina:** O raio simboliza a natureza inesperada e poderosa da mudança, muitas vezes além do nosso controle.
* **Perda e Libertação:** As figuras caindo e as coroas representam a perda de apegos materiais, poder e controle, levando a uma eventual libertação.

**Interpretações:**

* **Reviravoltas:** A Torre prenuncia mudanças abruptas e inesperadas na vida, como o fim de um relacionamento, perda de emprego ou crises pessoais.
* **Verdades Reveladas:** A carta expõe verdades ocultas, forçando-nos a confrontar a realidade e abandonar ilusões.
* **Transformação:** A Torre sinaliza um despertar espiritual, onde velhos padrões são quebrados para dar lugar a uma nova consciência.
* **Renovação:** Apesar do caos inicial, a 

In [255]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [256]:
prompt = f"Resuma esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**A Torre: A Carta do "Treme-Treme" no Tarô**

Imagina que você está numa torre alta, bem firme e segura. De repente, bum! Um raio cai e a torre desmorona. É assim que a carta da Torre no Tarô funciona.

**O que ela significa?**

* **Destruição:** Ela derruba tudo o que é velho e não serve mais, como crenças bobas e ilusões.
* **Mudança doida:** É como um terremoto, que vem do nada e bagunça tudo.
* **Perdas e liberdade:** Você pode perder coisas materiais ou poder, mas isso te liberta para coisas novas.

**Como ela afeta sua vida:**

* **Amor:** Pode acabar com relacionamentos ou revelar segredos.
* **Trabalho:** Pode te fazer perder o emprego ou mudar de carreira.
* **Dinheiro:** Pode te fazer perder grana ou te forçar a repensar seus investimentos.
* **Saúde:** Pode trazer doenças repentinas ou te fazer mudar seus hábitos.
* **Espiritualidade:** Pode te fazer acordar para a vida e te ajudar a encontrar um novo propósito.

**Como lidar com ela:**

* **Aceita:** Não lute contra a muda